In [90]:
import numpy as np
import random

class Structure:
    def __init__(self, seed, seed_next_possible):
        self.data = np.array([[0.0,0.0,0.0]])
        self.next_possibles = seed_next_possible
        self.rect = []
        self.rect.append(block_to_rect(seed))
        #print("initialized data: \n", self.data)

  
    def cost_func(self, pos):
        return random.random()
  
    def add_vertex(self, next_vertex):
        self.data = np.vstack([self.data, next_vertex])

    def remove_possible(self, next_index):
        self.next_possibles = np.delete(self.next_possibles, next_index, axis=0)
  
  #define direction:
  #x=-1 -> 0, x=1 -> 1, y=-1 -> 2, y=1 -> 3, z=-1 -> 4, z=1 -> 5,
    def get_next_possible(self, available_contact, direction):
        result = np.array([0,0,0,0])

        available_contact = np.append(available_contact, 0)
        if direction == 0 or direction == 1:
            result = np.vstack([result, available_contact + np.array([0.0,0.1,0.0,3])])
            result = np.vstack([result, available_contact + np.array([0.0,-0.1,0.0,2])])
            result = np.delete(result, 0, axis =0)
            return result

        if direction == 2 or direction == 3:
            result = np.vstack([result, available_contact + np.array([0.1,0.0,0.0,1])])
            result = np.vstack([result, available_contact + np.array([-0.1,0.0,0.0,1])])
            result = np.delete(result, 0, axis =0)
            return result

    
    def get_last_direction(self,direction):
        count =direction[1]*1 + direction[2]*2
        return count
  
    def get_vertex_forward(self, new_vertex_clean, direction):
        if direction == 0:
            return new_vertex_clean + np.array([-0.1,0.0,0.0])

        if direction == 1:
            return new_vertex_clean + np.array([0.1,0.0,0.0])

        if direction == 2:
            return new_vertex_clean + np.array([0.0,-0.1,0.0])

        if direction == 3:
            return new_vertex_clean + np.array([0.0,0.1,0.0])
    
    def generate(self, steps):
        for i in range(steps):
            #print("step ", i+1)
            self.process()
            self.rect.append(block_to_rect(self.data))
            self.data = np.array([[0,0,0]])



    def process(self):
        #step 1: for all available contacts, assume they will make a turn  
        #print("next_possibles", self.next_possibles)

        #step2: create a 1d array that stores cost for each possible point
        costs = np.array([])
        for next_possible in self.next_possibles:
            costs = np.append(costs, self.cost_func(next_possible))

        #print("costs", costs)

        #step3 find the next_vertex with smallest cost
        next_index = np.argpartition(costs, -1)[-1:]
        next_vertex = self.next_possibles[next_index] 

        #print("next_vertex", next_vertex)

        #step4 add next_vertex to data
        new_direction = next_vertex[0][3]
        #print("new_direction", new_direction)
        new_vertex_clean = np.array([next_vertex[0][0], next_vertex[0][1], next_vertex[0][2]])
        #print("new_vertex_clean", new_vertex_clean)
        self.add_vertex(new_vertex_clean)
        
        span = 2 + int(random.random() * 5 % 5)
        for i in range(span):
            new_vertex_clean = self.get_vertex_forward(new_vertex_clean, new_direction)
            self.add_vertex(new_vertex_clean)

        #print("data: \n", self.data)

        #step5 update next_possible
        self.remove_possible(next_index)
        #print("next_possibles, removed old", self.next_possibles)
        #print("next_vertex", new_vertex_clean)
        #print("new_direction", new_direction)
        new_next_possibles = self.get_next_possible(new_vertex_clean, new_direction)
        self.next_possibles = np.append(self.next_possibles,new_next_possibles, axis=0)
        #print("next_possibles, add new", self.next_possibles)


In [91]:
seed = np.array([[0.0,0.0,.00], [0.0,0.1,0.0],[0.0,0.2,0.0],[0.0,0.3,0.0], [0.0,0.4,0.0]])
seed_next_possible = np.array([[-0.1,0.4,0.0,0],[0.1,0.4,0.0,1]])


In [92]:
s = Structure(seed, seed_next_possible)

s.generate(5)

In [93]:
for i in s.rect:
    i.info()

start_x: 0.0 start_y: 0.1 start_z: 0.0
scale_x: 0.1 scale_y: 0.3 scale_z: 0.1
 
start_x: -0.5 start_y: 0.4 start_z: 0.0
scale_x: 0.4 scale_y: 0.1 scale_z: 0.1
 
start_x: 0.1 start_y: 0.4 start_z: 0.0
scale_x: 0.6 scale_y: 0.1 scale_z: 0.1
 
start_x: -0.5 start_y: -0.1 start_z: 0.0
scale_x: 0.1 scale_y: 0.4 scale_z: 0.1
 
start_x: -0.4 start_y: -0.1 start_z: 0.0
scale_x: 0.3 scale_y: 0.1 scale_z: 0.1
 
start_x: 0.7 start_y: 0.0 start_z: 0.0
scale_x: 0.1 scale_y: 0.3 scale_z: 0.1
 


In [88]:
def block_to_rect(buffer):
    start = buffer[1]
    end = buffer[-1]
    x_start = min(start[0],end[0])
    y_start = min(start[1],end[1])    
    z_start = min(start[2],end[2])
    
    x_scale = max(abs(start[0] - end[0]), 0.1)
    y_scale = max(abs(start[1] - end[1]), 0.1)
    z_scale = max(abs(start[2] - end[2]), 0.1)
    
    startPos = np.array([x_start,y_start,z_start])
    scale = np.array([x_scale,y_scale,z_scale])
    
    tempt = rect(startPos, scale)
    return tempt

In [89]:
class rect:
    def __init__(self, startPos, scale): 
        self.start_x = round(startPos[0],2);
        self.start_y = round(startPos[1],2);
        self.start_z = round(startPos[2],2);
        
        self.scale_x = round(scale[0],2);
        self.scale_y = round(scale[1],2);
        self.scale_z = round(scale[2],2);
    
    def info(self):
        print("start_x:",self.start_x,"start_y:",self.start_y,"start_z:",self.start_z)
        print("scale_x:",self.scale_x,"scale_y:",self.scale_y,"scale_z:",self.scale_z)
        print(" ")
    

In [81]:
b1 = np.array([[0,0,0], [0,1,0],[0,2,0],[0,3,0], [0,4,0]])
b2 = np.array([[0,0,1], [0,1,1],[0,2,1],[0,3,1], [0,4,1]])

In [82]:
rect_list = []

In [83]:
rect_list.append(block_to_rect(b1))
rect_list.append(block_to_rect(b2))
rect_list

[<__main__.rect at 0x7f89504b10d0>, <__main__.rect at 0x7f89504b1970>]